In [1]:
# dependencies
# !pip install gym box2d 

In [2]:
import os 
# os.environ['CUDA_VISIBLE_DEVICES'] = '4' # For Hex
# os.environ['KMP_DUPLICATE_LIB_OK']='True' # For Intel Mac

In [3]:
import time
import gym

In [4]:
### Import agent py file here
import DQN

In [5]:
dqn_agent = DQN.Agent(lr=0.001, gamma=0.99, epsilon=1.0, epsilon_decay=0.995, batch_size=64)

In [6]:
env = gym.make("LunarLander-v2")
spec = gym.spec("LunarLander-v2")


In [7]:
# ### TRAIN
# t_start = time.perf_counter()

# scores, avg_scores = dqn_agent.train_model(env, num_episodes)

# t_end = time.perf_counter()

# print(f"Finished in {t_end-t_start} seconds")

In [10]:
# TEST RENDER
num_episodes = 5

file_path = 'saved_networks/dqn_model_374_0.0001_0.99_es'
dqn_agent.test(env, num_episodes, file_path)

INFO:tensorflow:Assets written to: saved_networks/dqn_model_4_0.001_0.99_es/assets


OSError: Unable to open file (file read failed: time = Tue May 11 15:06:40 2021
, filename = 'saved_networks/dqn_model_374_0.0001_0.99_es', file descriptor = 60, errno = 21, error message = 'Is a directory', buf = 0x7ffee29f3c78, total read size = 8, bytes this sub-read = 8, bytes actually read = 18446744073709551615, offset = 0)